In [39]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix
from scipy import io
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint

In [88]:
FE.ReadHdf('test')

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238


id             uint32
Semana          uint8
Agencia_ID     uint16
Canal_ID        uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6999251 entries, 0 to 6999250
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 133.5 MB
None


In [91]:
FE.test.describe()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
count,6999251.000000,6999251.000000,6999251.000000,6999251.000000,6999251.000000,6.999251e+06,6999251.000000
mean,3499625.000000,10.494462,2504.462764,1.401874,2138.014097,1.819128e+06,22163.069696
std,2020509.868825,0.499969,4010.228289,1.513404,1500.391868,2.938910e+06,18698.156308
min,0.000000,10.000000,1110.000000,1.000000,1.000000,2.600000e+01,41.000000
25%,1749812.500000,10.000000,1311.000000,1.000000,1159.000000,3.558290e+05,1242.000000
50%,3499625.000000,10.000000,1612.000000,1.000000,1305.000000,1.200109e+06,31507.000000
75%,5249437.500000,11.000000,2034.000000,1.000000,2804.000000,2.387881e+06,40930.000000
max,6999250.000000,11.000000,25759.000000,11.000000,9950.000000,2.015152e+09,49997.000000


In [8]:
FE.test.describe()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
count,6999251.000000,6999251.000000,6999251.000000,6999251.000000,6999251.000000,6.999251e+06,6999251.000000
mean,3499625.000000,10.494462,2504.462764,1.401874,2138.014097,1.819128e+06,22163.069696
std,2020509.868825,0.499969,4010.228289,1.513404,1500.391868,2.938910e+06,18698.156308
min,0.000000,10.000000,1110.000000,1.000000,1.000000,2.600000e+01,41.000000
25%,1749812.500000,10.000000,1311.000000,1.000000,1159.000000,3.558290e+05,1242.000000
50%,3499625.000000,10.000000,1612.000000,1.000000,1305.000000,1.200109e+06,31507.000000
75%,5249437.500000,11.000000,2034.000000,1.000000,2804.000000,2.387881e+06,40930.000000
max,6999250.000000,11.000000,25759.000000,11.000000,9950.000000,2.015152e+09,49997.000000


In [10]:
FE.test.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [9]:
groupCount = FE.test.loc[:,['Semana','Producto_ID','Cliente_ID']+
              ['Demanda_uni_equil']].groupby(['Semana','Producto_ID','Cliente_ID']).count()
groupCount[groupCount['Demanda_uni_equil']>1].count()

Demanda_uni_equil    0
dtype: int64

In [86]:
class FeatureEngineering:
    #ValidationStart = 0
    #lag = 0
    #trainHdfPath = ""
    #trainHdfFile = ""
    #testHdfPath = ""
    #testHdfFile = ""
    #train = pd.DataFrame()
    #test = pd.DataFrame()
    
    def __init__(self, ValidationStart, ValidationSmallerThan, trainHdfPath, trainHdfFile, testHdfPath, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationSmallerThan = ValidationSmallerThan
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath = testHdfPath
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def __changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        data.index = data.index.astype('uint32')
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"
            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_hdf(self.testHdfPath,self.testHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"  
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            #__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            #__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)

    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
                self.train.loc[:,column] =  self.train[column].apply(func).astype(localType)
            elif trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
                self.test.loc[:,column] =  self.test[column].apply(func).astype(localType)
            
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t')
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test.to_hdf(self.testHdfPath, self.testHdfFile, format='t')
        
    def AddDemandaGeneralMean(self):
        DemandaMeanWithoutLag = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart]
        self.train.loc[:,"DemandaGeneralMean"] = DemandaMeanWithoutLag['Demanda_uni_equil'].mean()
        self.train.loc[:,"DemandaGeneralMean"] = self.train.loc[:,"DemandaGeneralMean"].astype('float32')
        display(self.train)
        
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        lag = 0 if config.lag == 0 else config.lag-1
        tempData = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart - lag]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData.loc[:,'Semana'].apply(lambda x:x + config.lag)
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
        
        for name,groups in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)
                
                ##First, select columns not to fill memory..then groupby..
                
                groupedDataframe =  pd.DataFrame({name:tempData[groups+['Demanda_uni_equil']].groupby(groups)
                    ['Demanda_uni_equil'].mean().astype('float32')})
                #join..
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                                  right_index=True, how='left', sort=False,copy=False)
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable]), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable]), name]
                count = self.train.loc[self.train['Semana'] >= self.ValidationStart,config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                gc.collect()
                if count == 0:
                    break
        display(self.train)        
        gc.collect()
        return 
    def DeleteLaggedWeeksFromTrain(self):
        self.train = self.train.loc[self.train.loc[:,'Semana']>= 3 + self.maxLag]
        display(self.train)

In [87]:
parameterDict =       {"ValidationStart":6, 
 "ValidationSmallerThan":7,
   "maxLag":2,
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath":"../../input/test.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'trainCsvPath': '../../input/train.csv', 'testHdfPath': '../../input/test.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 6, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train.h5', 'ValidationSmallerThan': 7}


In [122]:
testDF = pd.DataFrame({"Semana":[3,3,4,4,4,5], "Producto_ID": [5,5,5,5,5,5], "Cliente_ID": [4,4,4,4,4,4], 
    "Demanda_uni_equil":[3,7,9,1,4,3], "Agencia_ID":[1100,1100,1100,1100,1100,1100]    })
testDF = testDF[["Semana", "Producto_ID", "Cliente_ID", "Agencia_ID", "Demanda_uni_equil"]]

In [61]:
groupby1st = testDF.groupby(["Semana","Producto_ID","Cliente_ID"]).agg(["count","mean","sum"])
groupby1st.columns =  groupby1st.columns.droplevel(0)
groupby1st = groupby1st.reset_index()
groupby1st.drop("Semana",axis=1, inplace=True)
groupby2nd = groupby1st.groupby(["Producto_ID","Cliente_ID"]).agg(["count","mean","sum"])

In [62]:
groupby2nd[[("mean","mean"),("sum","mean")]]

,,mean,sum
,,mean,mean
Producto_ID,Cliente_ID,,
5,4,4.222222,9


In [63]:
groupby2nd

count           mean                        sum       \
                       count mean sum count      mean        sum count mean   
Producto_ID Cliente_ID                                                        
5           4              3    2   6     3  4.222222  12.666667     3    9   

                            
                       sum  
Producto_ID Cliente_ID      
5           4           27

In [52]:
qwe = groupby2nd

In [64]:
groupby2nd.columns = groupby2nd.columns.droplevel(0)

In [65]:
groupby2nd

,,count,mean,sum,count,mean,sum,count,mean,sum
Producto_ID,Cliente_ID,,,,,,,,,
5,4,3,2,6,3,4.222222,12.666667,3,9,27


In [69]:
qwe = groupby2nd.iloc[:,[4,7]]
qwe

,,mean,mean
Producto_ID,Cliente_ID,,
5,4,4.222222,9


In [80]:
testDF

,Semana,Producto_ID,Cliente_ID,Demanda_uni_equil
0,3,5,4,3
1,3,5,4,7
2,4,5,4,9
3,4,5,4,1
4,4,5,4,4
5,5,5,4,3


In [82]:
result = testDF.apply(lambda x:x.nunique())

In [83]:
result

Semana               3
Producto_ID          1
Cliente_ID           1
Demanda_uni_equil    5
dtype: int64

In [9]:
testDF

,Semana,Producto_ID,Cliente_ID,Demanda_uni_equil
0,3,5,4,3
1,3,5,4,7
2,4,5,4,9
3,4,5,4,1
4,4,5,4,4
5,5,5,4,3


In [7]:
groups = testDF.groupby(["Producto_ID","Semana"])

In [22]:
pd.unique(testDF[['Semana', 'Producto_ID']].values)

array([(3, 5), (4, 5), (5, 5)], dtype=object)

In [31]:
%timeit pd.unique(testDF[['Semana', 'Producto_ID']].values)

1000 loops, best of 3: 409 µs per loop


In [38]:
%timeit testDF.groupby(['Semana', 'Producto_ID']).indices.keys()

1000 loops, best of 3: 351 µs per loop


In [42]:
testDF[testDF.Semana>4]

,Semana,Producto_ID,Cliente_ID,Demanda_uni_equil
5,5,5,4,3


In [47]:
groupedDataframe =  pd.DataFrame({"berker":testDF[['Semana', 'Producto_ID','Demanda_uni_equil']].groupby(['Semana', 'Producto_ID'])
    ['Demanda_uni_equil'].agg(["mean"]).astype('float32')})

ValueError: If using all scalar values, you must pass an index

In [45]:
groupedDataframe

,,berker
Semana,Producto_ID,
3,5,5.000000
4,5,4.666667
5,5,3.000000


In [65]:
a = testDF[['Semana', 'Producto_ID','Demanda_uni_equil']].groupby(['Semana', 'Producto_ID']).agg(
    {"Demanda_uni_equil":{ "a1": "mean", "a2":"count"}})

In [67]:
a.columns = a.columns.droplevel(0)


In [68]:
a

,,a1,a2
Semana,Producto_ID,,
3,5,5.000000,2
4,5,4.666667,3
5,5,3.000000,1


In [105]:
testDF

,Semana,Producto_ID,Cliente_ID,Demanda_uni_equil
0,3,5,4,3
1,3,5,4,7
2,4,5,4,9
3,4,5,4,1
4,4,5,4,4
5,5,5,4,3


In [100]:
groupby1st = testDF.groupby(["Semana","Producto_ID","Cliente_ID"]).agg("count")
groupby1st.columns = ["qwe"]

In [95]:
groupby1st

,,,qwe
Semana,Producto_ID,Cliente_ID,
3,5,4,2
4,5,4,3
5,5,4,1


In [101]:
groupby1st.reset_index(inplace=True)

In [102]:
groupby1st

,Semana,Producto_ID,Cliente_ID,qwe
0,3,5,4,2
1,4,5,4,3
2,5,5,4,1


In [103]:
groupby1st.drop("Semana",axis=1, inplace=True)
groupby1st

,Producto_ID,Cliente_ID,qwe
0,5,4,2
1,5,4,3
2,5,4,1


In [104]:

groupby2nd = groupby1st.groupby(["Producto_ID","Cliente_ID"]).mean()
groupby2nd

,,qwe
Producto_ID,Cliente_ID,
5,4,2


In [110]:
groups = ["Semana","Agencia_ID"]

In [123]:
groupedDataframe = testDF[groups+['Demanda_uni_equil']].groupby(groups).agg("count")
                #groupedDataframe.columns = groupedDataframe.columns.droplevel(0)
groupedDataframe.columns = ["countt"]

In [124]:
groupedDataframe.reset_index(inplace=True)
groupedDataframe.drop("Semana",axis=1, inplace=True)


In [127]:
groupedDataframe

,countt
Agencia_ID,
1100,2


In [126]:
groupedDataframe = groupedDataframe.groupby(groups[1:]).mean()
gc.collect()

310

In [129]:
testDF = testDF.merge( groupedDataframe, left_on=groups[1:],
                                  right_index=True, how='left', sort=False,copy=False)

In [137]:
testDF.iloc[4,0] = np.nan
testDF

,Semana,Producto_ID,Cliente_ID,Agencia_ID,Demanda_uni_equil,countt
0,NaN,5,4,1100,3,2
1,NaN,5,4,1100,7,2
2,4,5,4,1100,9,2
3,4,5,4,1100,1,2
4,NaN,5,4,1100,4,2
5,5,5,4,1100,3,2


In [141]:
testDF.loc[:,"Semana"] = testDF.loc[pd.isnull(testDF["Semana"]),"Producto_ID"]

In [140]:
testDF.loc[pd.isnull(testDF["Semana"]),"Producto_ID"]

0    5
1    5
4    5
Name: Producto_ID, dtype: int64

In [147]:
testDF.loc[ pd.isnull(testDF["Semana"]),"Demanda_uni_equil"]

2    9
3    1
5    3
Name: Demanda_uni_equil, dtype: int64

In [153]:
testDF.loc[pd.isnull(testDF["Semana"]),"Semana"] = testDF.loc[ pd.isnull(testDF["Semana"]),"Demanda_uni_equil"]

In [148]:
testDF.loc[:,"Semana"] 

0     5
1     5
2   NaN
3   NaN
4     5
5   NaN
Name: Semana, dtype: float64

In [154]:
testDF

,Semana,Producto_ID,Cliente_ID,Agencia_ID,Demanda_uni_equil,countt
0,3,5,4,1100,3,2
1,7,5,4,1100,7,2
2,9,5,4,1100,9,2
3,1,5,4,1100,1,2
4,4,5,4,1100,4,2
5,3,5,4,1100,3,2
